# Logistische Regression
Die Logistische Regression ist ein Regressionsverfahren, das dann Anwendung findet, wenn die abhängige Variable y nur zwei Zustände annehmen kann, also z.B. *0* oder *1*, *True* oder *False* etc. Ein wichtiges Anwendungsgebiet ist Predictive Maintenance. Obwohl es es sich um ein Regressionsverfahren handelt, wird die Logistische Regression auch als Klassifikationsmodell verwendet, indem ein Schwellenwert (Threshold, Cut-Off) definiert wird. Wird dieser überschritten, so sagt das Modell *1* oder *True* voraus, ansonsten *0* oder *False*.

## Beispiel 1
Wir erstellen einen einfachen Beispieldatensatz mit Daten über die Anzahl der Schuljahre und ob die befragten Personen Führungskraft sind oder nicht:

In [ ]:
import pandas as pd

df = pd.DataFrame({"Schuljahre":[8,9,13,13,8,10,10,9,10,9,11,8,15,13,10],
                  "IstFuehrungskraft": [0,0,1,1,0,1,0,0,0,0,1,0,1,0,0]})


df

Wir erstellen ein *glm*-Objekt (Generalized Linear Model). Dass es sich um eine Logistische Regression handeln soll, teilen wir der Funktion durch die Angabe *family=sm.families.Binomial* mit.

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

model = smf.glm("IstFuehrungskraft~Schuljahre", data=df, family=sm.families.Binomial()).fit()
model.params

$\beta_0$ ist also -1.252033, $\beta_1$ ist 0.152439. Wir können somit die Wahrscheinlichkeit, dass eine Person z.B. über 14 Schuljahre verfügt, wie folgt berechnen:

$\frac{ 1 }{ 1+e^{10.695596 - 0.933998x} } = 0,92$

Oder wir verwenden die Funktion *predict*:

In [ ]:
model.predict(pd.DataFrame({"Schuljahre":[14]}))

Plotten wir noch die Funktion:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

x = np.linspace(df.Schuljahre.min(), df.Schuljahre.max(), 100)
y = model.predict(pd.DataFrame({"Schuljahre":x}))

plt.scatter(df.Schuljahre, df.IstFuehrungskraft)
plt.plot(x,y)
plt.xlabel("Anzahl Schuljahre")
plt.ylabel("P(Führungskraft)")
plt.show()


Mit *seaborn* kann auch direkt ein Regressionsplot erstellt werden. Dazu setzt man die Option *logistic* auf *True*:

In [ ]:
import seaborn as sns
df = pd.DataFrame({"Schuljahre":[8,9,13,13,8,10,10,9,10,9,11,8,15,13,10],
                  "IstFuehrungskraft": [0,0,1,1,0,1,0,0,0,0,1,0,1,0,0]})
sns.set_style("dark")
plt.xlim(7, 16)
sns.regplot(x="Schuljahre", y="IstFuehrungskraft", data=df, logistic=True, ci=False,
           line_kws={"color":"red", "lw":4},
           scatter_kws={"alpha":0.2, "color":"k"})
plt.title("Logistische Regression", color="crimson", fontsize=16, fontstyle="italic")
#plt.ylabel(r"$p_{Führungskraft}$")
plt.show()

In [ ]:
model.predict(pd.DataFrame({"Schuljahre":[8]}))

## Beispiel 2: Shuttle (Challanger-Katastrophe)
Vorhersage, ob die verbaute Gummidichtung (O-Ring) bei der damals herrschenden Temperatur abdichtet. Wir laden den Datensatz *shuttle.csv* und führen eine Logistische Regression durch.

In [ ]:
import pandas as pd
import statsmodels.api as sms
import statsmodels.formula.api as sfm

url = "https://raw.githubusercontent.com/troescherw/datasets/master/shuttle.csv"
shuttle = pd.read_csv(url)

model = sfm.glm("r~temperature", data=shuttle, family=sms.families.Binomial()).fit()
model.summary()

Am Tag es Unglücks herrschten angeblich 31 Grad Fahrenheit (-0,6 Grad Celsius). Mit welcher Wahrscheinlichkeit war die Gummidichtung nicht funktionsfähig?

In [ ]:
model.predict(pd.DataFrame({"temperature":[31]}))

Erstellen wir noch einen Plot:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
x = np.linspace(shuttle.temperature.min(), shuttle.temperature.max(), 50)
y = model.predict(pd.DataFrame({"temperature": x}))

plt.scatter(shuttle.temperature, shuttle.r)
plt.plot(x,y)
plt.title("Wahrscheinlichkeit der Dichtigkeit des O-Rings von der Temperatur")
plt.xlabel("Temperatur in Grad Fahrenheit")
plt.ylabel("P(Undicht)")
plt.show()

## Beispiel 3: TITANIC
Überlebenswahrscheinlicheit auf der Titanic in Abhängigkeit von Geschlecht, Alter und Klasse.

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/troescherw/datasets/master/titanic.csv"
titanic = pd.read_csv(url)
titanic

Wir wollen die Überlebenswahrscheinlichkeit (*Survived*) in Abhängigkeit von Alter (*Age*), Geschlecht (*Sex*) und der Klasse (*Pclass*) vorhersagen. Wir überprüfen zuersat, ob es NaNs im Datensatz gibt:

In [ ]:
titanic.isna().sum()

Bei *Age* gibt es 177 NaNs! Wir ersetzen diese durch den Mittelwert der Altersangaben.

In [ ]:
mean_age = titanic.Age.mean()
titanic.Age = titanic.Age.fillna(mean_age)

Nun teilen wir den Datensatz in einen Trainings- und Testdatensatz auf.70% verwenden wir für das Training, 30% für das Testen.



In [ ]:
from sklearn.model_selection import train_test_split
X = titanic[["Survived", "Age", "Sex", "Pclass"]]
y = titanic.Survived

X_train, X_test, y_train, y_test = train_test_split(X,y,shuffle=True, random_state=42, test_size=0.3)

Nun erstellen wir das Vorhersagemodell, erstellen anhand der Test-Daten eine Prognose und stellen das Ergebnis in einer Confusion-Matrix dar. Beim Geschlecht und bei der Klasse handelt es sich um kategoriale Variablen, daher setzen wir in der Formel diese Kategorien in Klammern hinter einem *C*.

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

model = smf.glm("Survived~Age+C(Sex)+C(Pclass)", data=X_train, family=sm.families.Binomial()).fit()
model.summary()

Die Vorhersage liefert uns die Wahrscheinlichkeiten. Wir setzen einen Cut-Off bei 0,5: Bei einer Überlebenswahrscheinlichkeit >0,5 setzen wir eine *1* (Survived), ansonsten eine *0*.

In [ ]:
from sklearn.metrics import classification_report

pred = model.predict(X_test)
pred_class =pd.Series([1 if x >0.5 else 0 for x in pred])

print(classification_report(y_test, pred_class))

Wir erreichen also eine **Accuracy** von 81%.

Sagen wir die Überlebenswahrscheinlichkeit für einen männlichen Passagier, 20 Jahre in der 3. Klasse voraus:

In [ ]:
model.predict(pd.DataFrame({"Age": [20], "Sex":["male"], "Pclass" : 3}))

Erstellen wir noch einen Plot mit der Überlebenswahrscheinlichkeit in Abhängigkeit von Alter und Geschlecht der Passagiere der 1. Klasse:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = np.linspace(titanic.Age.min(), titanic.Age.max(), 50)
y_male = model.predict(pd.DataFrame({"Sex":"male", "Age":x, "Pclass": 1}))
y_female = model.predict(pd.DataFrame({"Sex":"female", "Age":x, "Pclass": 1}))

plt.plot(x, y_male, label="Männer")
plt.plot(x, y_female, label="Frauen")
plt.legend()#
plt.xlabel("Alter")
plt.ylabel("Überlebenswahrscheinlichkeit")
plt.title("Überlebenswahrscheinlichkeit der Männer und Frauen der 1. Klasse")
plt.show()

## ROC-Kurve
Erstellen wir noch eine ROC-Kurve für unser Modell und geben die AUC aus.

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_test, pred)

plt.plot(fpr, tpr)
plt.title("ROC-Kurve")
plt.xlabel("fpr")
plt.ylabel("tpr")
plt.plot()

print("AUC = ", roc_auc_score(y_test, pred_class))

## Beispiel 4: Vorhersage einer Herzkrankheit
Ein Beispiel aus der Medizin! Wir wollen anhand des Datensatzes *heartdisease.csv* mit den folgenden Features vorhersagen, ob eine Person an einer Herzkrankheit erkranken wird:

**Beschreibung der Features:**

|Feature | Bedeutung |
|:-------|:----------|
|**age**| age in years|
|**sex**| sex (1 = male; 0 = female)|
|**cp**| chest pain type|
|| Value 1 : typical angina|
|| Value 2 : atypical angina|
|| Value 3 : non-anginal pain|
|| Value 4 : asymptomatic|
| **trestbps**| resting blood pressure (in mm Hg on admission to the hospital)|
| **chol**| serum cholestoral in mg/dl|
| **fbs**| (fasting blood sugar > 120 mg/dl)  (1 = true; 0 = false)|
| **restecg**| resting electrocardiographic results|
|| Value 0: normal|
|| Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)|
|| Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria|
| **thalach**| maximum heart rate achieved|
| **exang**| exercise induced angina (1 = yes; 0 = no)|
| **oldpeak** | ST depression induced by exercise relative to rest|
| **slope**| the slope of the peak exercise ST segment|
|| Value 1: upsloping|
|| Value 2: flat|
|| Value 3: downsloping|
| **ca**| number of major vessels (0-3) colored by flourosopy|
| **thal**| 3 = normal; 6 = fixed defect; 7 = reversable defect|
| **hd**| **diagnosis of heart disease**| 

Wir laden den Datensatz in ein Pandas DataFrame. Da das Feature *hd* nicht nur angibt, ob, sondern auch welche Herzerkrankung vorliegt, setzen wir alle Werte >=1 auf 1.

In [ ]:
import pandas as pd
url = "https://raw.githubusercontent.com/troescherw/datasets/master/heartdisease.csv"
headers = ["age","sex","cp","trestbps","chol","fbs","restecg","thalach","exang","oldpeak","slope","ca","thal","status"]
df = pd.read_csv(url, header=None)
df.columns=headers
df.status = ["krank" if x>=1 else "gesund" for x in df.status]
print(df.shape)
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

X = df
y = df.status

# Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X,y,shuffle=True, random_state=0)

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.metrics import classification_report

# Modell erstellen
# cp, restecg und slope sind kategoriale Variablen
X = sm.add_constant(X)
model = smf.glm("C(status)~age+sex+C(cp)+trestbps+chol+fbs+C(restecg)+thalach+exang+oldpeak+C(slope)+ca+thal", 
                data=X_train, family=sm.families.Binomial()).fit()

# Vorhersage mit Testdaten
pred = model.predict(X_test)
pred_class = ["gesund" if x >0.5 else "krank" for x in pred]

# Modellreport
print(classification_report(pred_class, y_test))

In [ ]:
model.summary()

Erstellen wir noch eine ROC-Kurve und lassen die AUC berechnen:

In [ ]:
from sklearn.metrics import roc_curve


y_test = [1 if x=="gesund" else 0 for x in y_test]
fpr, tpr, thresholds = roc_curve(y_test, pred)


plt.plot(fpr, tpr)
plt.title("ROC-Kurve")
plt.xlabel("fpr")
plt.ylabel("tpr")
plt.plot()

In [ ]:
from sklearn.metrics import roc_auc_score
pred_class = [1 if x=="gesund" else 0 for x in pred_class]
print("AUC = ", roc_auc_score(y_test, pred_class))